<a href="https://colab.research.google.com/github/shrinidhi1511/project1/blob/main/foodprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# app.py
"""
Digital Twin Hackathon Demo App
Features:
 - Simple login (operator / manager)
 - Live simulated sensor stream (start / stop)
 - Virtual control panel (setpoints affect simulated values)
 - Upload CSV / JSON to feed dashboard instead of simulation
 - Role-based UI: operator vs manager
 - Downloadable batch reports (manager)
"""

import streamlit as st
import pandas as pd
import numpy as np
import time
from io import StringIO

# -------------------------
# --------- CONFIG --------
# -------------------------
USERS = {
    "operator": {"password": "op123", "role": "operator"},
    "manager": {"password": "mg123", "role": "manager"}
}

SIM_BATCH_SIZE = 20  # number of records per batch (batch_id increments every SIM_BATCH_SIZE rows)

# -------------------------
# ---- Helper functions ---
# -------------------------
def authenticate(username: str, password: str):
    user = USERS.get(username)
    if user and user["password"] == password:
        return {"username": username, "role": user["role"]}
    return None

def init_session():
    if "logged_in" not in st.session_state:
        st.session_state.logged_in = False
        st.session_state.user = None
    if "data" not in st.session_state:
        st.session_state.data = pd.DataFrame(columns=["timestamp","temp","humidity","vibration","batch_id"])
    if "running" not in st.session_state:
        st.session_state.running = False
    if "sim_index" not in st.session_state:
        st.session_state.sim_index = 0
    if "setpoint_temp" not in st.session_state:
        st.session_state.setpoint_temp = 60.0
        st.session_state.setpoint_humidity = 50.0
        st.session_state.setpoint_vibration = 0.5

def generate_record(i, setpoints):
    """Simulate one sensor record influenced by setpoints."""
    base_temp = setpoints["temp"]
    base_hum = setpoints["humidity"]
    base_vib = setpoints["vibration"]

    # add small random fluctuations, and occasional drift
    temp = np.random.normal(base_temp, 2.0) + 0.5 * np.sin(i/10)
    humidity = np.random.normal(base_hum, 1.5) + 0.2 * np.cos(i/13)
    vibration = max(0.01, np.random.normal(base_vib, 0.05) + 0.02 * np.sin(i/7))

    batch_id = f"batch_{(i // SIM_BATCH_SIZE) + 1}"
    return {
        "timestamp": pd.Timestamp.now(),
        "temp": round(float(temp), 2),
        "humidity": round(float(humidity), 2),
        "vibration": round(float(vibration), 3),
        "batch_id": batch_id
    }

def append_record(rec):
    df = st.session_state.data
    df = pd.concat([df, pd.DataFrame([rec])], ignore_index=True)
    st.session_state.data = df

def load_uploaded_file(uploaded_file):
    try:
        file_ext = uploaded_file.name.split(".")[-1].lower()
        if file_ext in ("csv", "txt"):
            df = pd.read_csv(uploaded_file)
        elif file_ext in ("json",):
            df = pd.read_json(uploaded_file)
        else:
            st.error("Unsupported file type. Please upload CSV or JSON.")
            return None

        # Expect at least columns: timestamp,temp,humidity,vibration,batch_id
        # If timestamp missing, create one.
        if "timestamp" not in df.columns:
            df["timestamp"] = pd.Timestamp.now()
        required = ["timestamp","temp","humidity","vibration"]
        for c in required:
            if c not in df.columns:
                st.error(f"Uploaded file missing required column: {c}")
                return None

        # Normalize columns and types
        df = df[["timestamp","temp","humidity","vibration"]].copy()
        if "batch_id" in uploaded_file.name:  # optional way to specify batches via filename
            df["batch_id"] = uploaded_file.name.split(".")[0]
        else:
            # create batch ids if not present
            df["batch_id"] = [f"batch_{(i // SIM_BATCH_SIZE) + 1}" for i in range(len(df))]

        # coerce timestamp
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        return df
    except Exception as e:
        st.error(f"Failed to parse uploaded file: {e}")
        return None

# -------------------------
# ------- UI LAYOUT -------
# -------------------------
st.set_page_config(page_title="Digital Twin Dashboard", layout="wide")
init_session()

if not st.session_state.logged_in:
    st.title("🔐 Login")
    col1, col2 = st.columns(2)
    with col1:
        username = st.text_input("Username")
    with col2:
        password = st.text_input("Password", type="password")

    if st.button("Login"):
        auth = authenticate(username, password)
        if auth:
            st.session_state.logged_in = True
            st.session_state.user = auth
            st.experimental_rerun()
        else:
            st.error("Invalid username or password. Try 'operator/op123' or 'manager/mg123'.")

else:
    user = st.session_state.user
    st.sidebar.title(f"Welcome, {user['username']} ({user['role']})")
    if st.sidebar.button("Logout"):
        # clear session and rerun
        for k in list(st.session_state.keys()):
            del st.session_state[k]
        st.experimental_rerun()

    # Navigation
    page = st.sidebar.radio("Page", ["Dashboard", "Control Panel", "Upload Data", "Admin" if user["role"]=="manager" else "Info"])

    # Top-level summary
    st.markdown("## 🍪 Digital Twin — Food Process Demo")
    st.markdown("**Live mode:** Simulated data. You can upload real CSV/JSON to switch to offline mode.")

    # Right-side container for charts & table
    if page == "Dashboard":
        left, right = st.columns([3,1])

        # Controls for simulation
        with right:
            st.subheader("Simulation Controls")
            if not st.session_state.running:
                if st.button("Start Simulation"):
                    st.session_state.running = True
            else:
                if st.button("Stop Simulation"):
                    st.session_state.running = False

            st.markdown("---")
            st.write("Setpoints (affect simulation)")
            st.session_state.setpoint_temp = st.slider("Target Temp (°C)", 20.0, 120.0, st.session_state.setpoint_temp)
            st.session_state.setpoint_humidity = st.slider("Target Humidity (%)", 0.0, 100.0, st.session_state.setpoint_humidity)
            st.session_state.setpoint_vibration = st.slider("Target Vibration (g)", 0.0, 5.0, st.session_state.setpoint_vibration)
            st.markdown("---")
            st.write("Data source:")
            st.write("• If you've uploaded data, the app will display that. Otherwise the simulation runs.")
            if st.button("Clear Stored Data"):
                st.session_state.data = pd.DataFrame(columns=["timestamp","temp","humidity","vibration","batch_id"])
                st.success("Cleared stored data")

        # Main charts
        with left:
            st.subheader("Live Sensor Chart")
            chart_placeholder = st.empty()

            st.subheader("Latest readings")
            data_table = st.empty()

            # Insert uploaded data if present in session_state (upload page will fill it)
            df = st.session_state.data.copy()

            # If simulation running, generate a new record per rerun
            if st.session_state.running:
                # generate a single record per rerun (Streamlit reruns the script frequently)
                rec = generate_record(st.session_state.sim_index, {
                    "temp": st.session_state.setpoint_temp,
                    "humidity": st.session_state.setpoint_humidity,
                    "vibration": st.session_state.setpoint_vibration
                })
                append_record(rec)
                st.session_state.sim_index += 1
                # tiny pause so UI updates look natural (non-blocking enough)
                time.sleep(0.25)

            if df.empty:
                st.info("No data yet. Start simulation or upload a file.")
            else:
                # ensure timestamp is datetime and sorted
                df["timestamp"] = pd.to_datetime(df["timestamp"])
                df = df.sort_values("timestamp")
                df2 = df.set_index("timestamp")[["temp","humidity","vibration"]].tail(300)

                # show line chart
                chart_placeholder.line_chart(df2)

                # Latest table
                data_table.dataframe(df.tail(10).reset_index(drop=True))

            # Batch status: show last value per batch
            st.subheader("Batch Status")
            if not df.empty:
                last_per_batch = df.sort_values("timestamp").groupby("batch_id").tail(1)
                st.table(last_per_batch[["batch_id","temp","humidity","vibration"]].reset_index(drop=True))

    elif page == "Control Panel":
        st.subheader("Virtual Control Panel")
        st.write("This panel simulates remote adjustments to the physical process. Changes affect subsequent simulated data.")
        st.markdown("- Adjust setpoints and then Start Simulation to observe the effect in the Dashboard.")
        st.write("Current setpoints:")
        st.write(f"Temperature: {st.session_state.setpoint_temp} °C")
        st.write(f"Humidity: {st.session_state.setpoint_humidity} %")
        st.write(f"Vibration: {st.session_state.setpoint_vibration} g")

        if user["role"] != "operator":
            st.info("Control access is normally for operators. Managers can view and adjust for testing here.")

        # quick single-step controls (for demos)
        if st.button("Simulate Heat Spike (+10°C) for 10 records"):
            base_t = st.session_state.setpoint_temp
            for j in range(10):
                rec = generate_record(st.session_state.sim_index + j, {
                    "temp": base_t + 10,
                    "humidity": st.session_state.setpoint_humidity,
                    "vibration": st.session_state.setpoint_vibration
                })
                append_record(rec)
            st.session_state.sim_index += 10
            st.success("Injected heat spike (10 records)")

    elif page == "Upload Data":
        st.subheader("Upload CSV or JSON sensor data")
        st.markdown("Expected columns: timestamp, temp, humidity, vibration (optional: batch_id). If timestamp absent, it will be added.")
        uploaded_file = st.file_uploader("Upload file", type=["csv","json"])
        if uploaded_file is not None:
            df_uploaded = load_uploaded_file(uploaded_file)
            if df_uploaded is not None:
                st.session_state.data = pd.concat([st.session_state.data, df_uploaded], ignore_index=True)
                st.success(f"Loaded {len(df_uploaded)} rows from upload into session storage.")
                st.experimental_rerun()

    elif page == "Admin":
        st.subheader("Manager Analytics & Export")
        df = st.session_state.data.copy()
        if df.empty:
            st.info("No data to analyze. Start simulation or upload a file.")
        else:
            df["timestamp"] = pd.to_datetime(df["timestamp"])
            st.markdown("**Summary statistics (latest data)**")
            st.write(df[["temp","humidity","vibration"]].describe())

            # simple anomaly rule: temp > setpoint + 10 or vibration > 1.5
            df["anomaly_rule"] = ((df["temp"] > (st.session_state.setpoint_temp + 10)) | (df["vibration"] > 1.5)).astype(int)
            st.markdown("Anomaly counts by batch (rule-based)")
            anom_counts = df.groupby("batch_id")["anomaly_rule"].sum().reset_index().rename(columns={"anomaly_rule":"anomaly_count"})
            st.table(anom_counts)

            st.markdown("Download combined data (CSV)")
            csv = df.to_csv(index=False)
            st.download_button("Download CSV", data=csv, file_name="sensor_data_export.csv", mime="text/csv")

    else:
        st.subheader("Info")
        st.markdown("This demo app simulates a digital twin pipeline. Use the Upload page to test with real data. Use Control Panel to inject conditions and Dashboard to view live charts.")

    # footer: small help
    st.markdown("---")
    st.caption("Demo app: operator credentials — user: operator / pass: op123. manager — user: manager / pass: mg123")


2025-09-18 07:22:12.763 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 07:22:12.764 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 07:22:12.766 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 07:22:12.767 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 07:22:12.768 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 07:22:12.769 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 07:22:12.772 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 07:22:12.774 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [11]:
# app.py
"""
Digital Twin Hackathon Demo App
Features:
 - Simple login (operator / manager)
 - Live simulated sensor stream (start / stop)
 - Virtual control panel (setpoints affect simulated values)
 - Upload CSV / JSON to feed dashboard instead of simulation
 - Role-based UI: operator vs manager
 - Downloadable batch reports (manager)
"""

import streamlit as st
import pandas as pd
import numpy as np
import time
from io import StringIO

# -------------------------
# --------- CONFIG --------
# -------------------------
USERS = {
    "operator": {"password": "op123", "role": "operator"},
    "manager": {"password": "mg123", "role": "manager"}
}

SIM_BATCH_SIZE = 20  # number of records per batch (batch_id increments every SIM_BATCH_SIZE rows)

# -------------------------
# ---- Helper functions ---
# -------------------------
def authenticate(username: str, password: str):
    user = USERS.get(username)
    if user and user["password"] == password:
        return {"username": username, "role": user["role"]}
    return None

def init_session():
    if "logged_in" not in st.session_state:
        st.session_state.logged_in = False
        st.session_state.user = None
    if "data" not in st.session_state:
        st.session_state.data = pd.DataFrame(columns=["timestamp","temp","humidity","vibration","batch_id"])
    if "running" not in st.session_state:
        st.session_state.running = False
    if "sim_index" not in st.session_state:
        st.session_state.sim_index = 0
    if "setpoint_temp" not in st.session_state:
        st.session_state.setpoint_temp = 60.0
        st.session_state.setpoint_humidity = 50.0
        st.session_state.setpoint_vibration = 0.5

def generate_record(i, setpoints):
    """Simulate one sensor record influenced by setpoints."""
    base_temp = setpoints["temp"]
    base_hum = setpoints["humidity"]
    base_vib = setpoints["vibration"]

    # add small random fluctuations, and occasional drift
    temp = np.random.normal(base_temp, 2.0) + 0.5 * np.sin(i/10)
    humidity = np.random.normal(base_hum, 1.5) + 0.2 * np.cos(i/13)
    vibration = max(0.01, np.random.normal(base_vib, 0.05) + 0.02 * np.sin(i/7))

    batch_id = f"batch_{(i // SIM_BATCH_SIZE) + 1}"
    return {
        "timestamp": pd.Timestamp.now(),
        "temp": round(float(temp), 2),
        "humidity": round(float(humidity), 2),
        "vibration": round(float(vibration), 3),
        "batch_id": batch_id
    }

def append_record(rec):
    df = st.session_state.data
    df = pd.concat([df, pd.DataFrame([rec])], ignore_index=True)
    st.session_state.data = df

def load_uploaded_file(uploaded_file):
    try:
        file_ext = uploaded_file.name.split(".")[-1].lower()
        if file_ext in ("csv", "txt"):
            df = pd.read_csv(uploaded_file)
        elif file_ext in ("json",):
            df = pd.read_json(uploaded_file)
        else:
            st.error("Unsupported file type. Please upload CSV or JSON.")
            return None

        # Expect at least columns: timestamp,temp,humidity,vibration,batch_id
        # If timestamp missing, create one.
        if "timestamp" not in df.columns:
            df["timestamp"] = pd.Timestamp.now()
        required = ["timestamp","temp","humidity","vibration"]
        for c in required:
            if c not in df.columns:
                st.error(f"Uploaded file missing required column: {c}")
                return None

        # Normalize columns and types
        df = df[["timestamp","temp","humidity","vibration"]].copy()
        if "batch_id" in uploaded_file.name:  # optional way to specify batches via filename
            df["batch_id"] = uploaded_file.name.split(".")[0]
        else:
            # create batch ids if not present
            df["batch_id"] = [f"batch_{(i // SIM_BATCH_SIZE) + 1}" for i in range(len(df))]

        # coerce timestamp
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        return df
    except Exception as e:
        st.error(f"Failed to parse uploaded file: {e}")
        return None

# -------------------------
# ------- UI LAYOUT -------
# -------------------------
st.set_page_config(page_title="Digital Twin Dashboard", layout="wide")
init_session()

if not st.session_state.logged_in:
    st.title("🔐 Login")
    col1, col2 = st.columns(2)
    with col1:
        username = st.text_input("Username")
    with col2:
        password = st.text_input("Password", type="password")

    if st.button("Login"):
        auth = authenticate(username, password)
        if auth:
            st.session_state.logged_in = True
            st.session_state.user = auth
            st.experimental_rerun()
        else:
            st.error("Invalid username or password. Try 'operator/op123' or 'manager/mg123'.")

else:
    user = st.session_state.user
    st.sidebar.title(f"Welcome, {user['username']} ({user['role']})")
    if st.sidebar.button("Logout"):
        # clear session and rerun
        for k in list(st.session_state.keys()):
            del st.session_state[k]
        st.experimental_rerun()

    # Navigation
    page = st.sidebar.radio("Page", ["Dashboard", "Control Panel", "Upload Data", "Admin" if user["role"]=="manager" else "Info"])

    # Top-level summary
    st.markdown("## 🍪 Digital Twin — Food Process Demo")
    st.markdown("**Live mode:** Simulated data. You can upload real CSV/JSON to switch to offline mode.")

    # Right-side container for charts & table
    if page == "Dashboard":
        left, right = st.columns([3,1])

        # Controls for simulation
        with right:
            st.subheader("Simulation Controls")
            if not st.session_state.running:
                if st.button("Start Simulation"):
                    st.session_state.running = True
            else:
                if st.button("Stop Simulation"):
                    st.session_state.running = False

            st.markdown("---")
            st.write("Setpoints (affect simulation)")
            st.session_state.setpoint_temp = st.slider("Target Temp (°C)", 20.0, 120.0, st.session_state.setpoint_temp)
            st.session_state.setpoint_humidity = st.slider("Target Humidity (%)", 0.0, 100.0, st.session_state.setpoint_humidity)
            st.session_state.setpoint_vibration = st.slider("Target Vibration (g)", 0.0, 5.0, st.session_state.setpoint_vibration)
            st.markdown("---")
            st.write("Data source:")
            st.write("• If you've uploaded data, the app will display that. Otherwise the simulation runs.")
            if st.button("Clear Stored Data"):
                st.session_state.data = pd.DataFrame(columns=["timestamp","temp","humidity","vibration","batch_id"])
                st.success("Cleared stored data")

        # Main charts
        with left:
            st.subheader("Live Sensor Chart")
            chart_placeholder = st.empty()

            st.subheader("Latest readings")
            data_table = st.empty()

            # Insert uploaded data if present in session_state (upload page will fill it)
            df = st.session_state.data.copy()

            # If simulation running, generate a new record per rerun
            if st.session_state.running:
                # generate a single record per rerun (Streamlit reruns the script frequently)
                rec = generate_record(st.session_state.sim_index, {
                    "temp": st.session_state.setpoint_temp,
                    "humidity": st.session_state.setpoint_humidity,
                    "vibration": st.session_state.setpoint_vibration
                })
                append_record(rec)
                st.session_state.sim_index += 1
                # tiny pause so UI updates look natural (non-blocking enough)
                time.sleep(0.25)

            if df.empty:
                st.info("No data yet. Start simulation or upload a file.")
            else:
                # ensure timestamp is datetime and sorted
                df["timestamp"] = pd.to_datetime(df["timestamp"])
                df = df.sort_values("timestamp")
                df2 = df.set_index("timestamp")[["temp","humidity","vibration"]].tail(300)

                # show line chart
                chart_placeholder.line_chart(df2)

                # Latest table
                data_table.dataframe(df.tail(10).reset_index(drop=True))

            # Batch status: show last value per batch
            st.subheader("Batch Status")
            if not df.empty:
                last_per_batch = df.sort_values("timestamp").groupby("batch_id").tail(1)
                st.table(last_per_batch[["batch_id","temp","humidity","vibration"]].reset_index(drop=True))

    elif page == "Control Panel":
        st.subheader("Virtual Control Panel")
        st.write("This panel simulates remote adjustments to the physical process. Changes affect subsequent simulated data.")
        st.markdown("- Adjust setpoints and then Start Simulation to observe the effect in the Dashboard.")
        st.write("Current setpoints:")
        st.write(f"Temperature: {st.session_state.setpoint_temp} °C")
        st.write(f"Humidity: {st.session_state.setpoint_humidity} %")
        st.write(f"Vibration: {st.session_state.setpoint_vibration} g")

        if user["role"] != "operator":
            st.info("Control access is normally for operators. Managers can view and adjust for testing here.")

        # quick single-step controls (for demos)
        if st.button("Simulate Heat Spike (+10°C) for 10 records"):
            base_t = st.session_state.setpoint_temp
            for j in range(10):
                rec = generate_record(st.session_state.sim_index + j, {
                    "temp": base_t + 10,
                    "humidity": st.session_state.setpoint_humidity,
                    "vibration": st.session_state.setpoint_vibration
                })
                append_record(rec)
            st.session_state.sim_index += 10
            st.success("Injected heat spike (10 records)")

    elif page == "Upload Data":
        st.subheader("Upload CSV or JSON sensor data")
        st.markdown("Expected columns: timestamp, temp, humidity, vibration (optional: batch_id). If timestamp absent, it will be added.")
        uploaded_file = st.file_uploader("Upload file", type=["csv","json"])
        if uploaded_file is not None:
            df_uploaded = load_uploaded_file(uploaded_file)
            if df_uploaded is not None:
                st.session_state.data = pd.concat([st.session_state.data, df_uploaded], ignore_index=True)
                st.success(f"Loaded {len(df_uploaded)} rows from upload into session storage.")
                st.experimental_rerun()

    elif page == "Admin":
        st.subheader("Manager Analytics & Export")
        df = st.session_state.data.copy()
        if df.empty:
            st.info("No data to analyze. Start simulation or upload a file.")
        else:
            df["timestamp"] = pd.to_datetime(df["timestamp"])
            st.markdown("**Summary statistics (latest data)**")
            st.write(df[["temp","humidity","vibration"]].describe())

            # simple anomaly rule: temp > setpoint + 10 or vibration > 1.5
            df["anomaly_rule"] = ((df["temp"] > (st.session_state.setpoint_temp + 10)) | (df["vibration"] > 1.5)).astype(int)
            st.markdown("Anomaly counts by batch (rule-based)")
            anom_counts = df.groupby("batch_id")["anomaly_rule"].sum().reset_index().rename(columns={"anomaly_rule":"anomaly_count"})
            st.table(anom_counts)

            st.markdown("Download combined data (CSV)")
            csv = df.to_csv(index=False)
            st.download_button("Download CSV", data=csv, file_name="sensor_data_export.csv", mime="text/csv")

    else:
        st.subheader("Info")
        st.markdown("This demo app simulates a digital twin pipeline. Use the Upload page to test with real data. Use Control Panel to inject conditions and Dashboard to view live charts.")

    # footer: small help
    st.markdown("---")
    st.caption("Demo app: operator credentials — user: operator / pass: op123. manager — user: manager / pass: mg123")


2025-09-18 09:53:24.263 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 09:53:24.264 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 09:53:24.265 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 09:53:24.267 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 09:53:24.268 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 09:53:24.269 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 09:53:24.274 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 09:53:24.276 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [18]:
# Install required packages
!pip install streamlit pyngrok --quiet

# Save your Streamlit app to a file
app_code = """
import streamlit as st
import pandas as pd
import numpy as np
import time

USERS = {"operator": {"password":"op123","role":"operator"},
         "manager": {"password":"mg123","role":"manager"}}

SIM_BATCH_SIZE = 20

def authenticate(username, password):
    user = USERS.get(username)
    if user and user["password"] == password:
        return {"username": username, "role": user["role"]}
    return None

if "logged_in" not in st.session_state:
    st.session_state.logged_in = False
    st.session_state.user = None
    st.session_state.data = pd.DataFrame(columns=["timestamp","temp","humidity","vibration","batch_id"])
    st.session_state.running = False
    st.session_state.sim_index = 0
    st.session_state.setpoint_temp = 60.0
    st.session_state.setpoint_humidity = 50.0
    st.session_state.setpoint_vibration = 0.5

def generate_record(i, setpoints):
    temp = np.random.normal(setpoints["temp"],2.0)+0.5*np.sin(i/10)
    humidity = np.random.normal(setpoints["humidity"],1.5)+0.2*np.cos(i/13)
    vibration = max(0.01, np.random.normal(setpoints["vibration"],0.05)+0.02*np.sin(i/7))
    batch_id = f"batch_{(i // SIM_BATCH_SIZE)+1}"
    return {"timestamp":pd.Timestamp.now(), "temp":round(float(temp),2),
            "humidity":round(float(humidity),2), "vibration":round(float(vibration),3),
            "batch_id":batch_id}

def append_record(rec):
    df = st.session_state.data
    st.session_state.data = pd.concat([df,pd.DataFrame([rec])],ignore_index=True)

# --- LOGIN ---
if not st.session_state.logged_in:
    st.title("🔐 Login")
    username = st.text_input("Username")
    password = st.text_input("Password", type="password")
    if st.button("Login"):
        auth = authenticate(username, password)
        if auth:
            st.session_state.logged_in = True
            st.session_state.user = auth
            st.experimental_rerun()
        else:
            st.error("Invalid username or password. Try 'operator/op123' or 'manager/mg123'.")
else:
    user = st.session_state.user
    st.sidebar.title(f"Welcome, {user['username']} ({user['role']})")
    if st.sidebar.button("Logout"):
        for k in list(st.session_state.keys()):
            del st.session_state[k]
        st.experimental_rerun()

    st.markdown("## 🍪 Digital Twin Demo")
    left, right = st.columns([3,1])

    # Simulation controls
    with right:
        if not st.session_state.running:
            if st.button("Start Simulation"):
                st.session_state.running = True
        else:
            if st.button("Stop Simulation"):
                st.session_state.running = False
        st.session_state.setpoint_temp = st.slider("Temp (°C)",20.0,120.0,st.session_state.setpoint_temp)
        st.session_state.setpoint_humidity = st.slider("Humidity (%)",0.0,100.0,st.session_state.setpoint_humidity)
        st.session_state.setpoint_vibration = st.slider("Vibration (g)",0.0,5.0,st.session_state.setpoint_vibration)

    # Dashboard
    with left:
        chart_placeholder = st.empty()
        data_table = st.empty()
        df = st.session_state.data.copy()

        if st.session_state.running:
            rec = generate_record(st.session_state.sim_index, {
                "temp": st.session_state.setpoint_temp,
                "humidity": st.session_state.setpoint_humidity,
                "vibration": st.session_state.setpoint_vibration
            })
            append_record(rec)
            st.session_state.sim_index += 1
            time.sleep(0.25)

        if df.empty:
            st.info("No data yet. Start simulation.")
        else:
            df = df.sort_values("timestamp")
            df2 = df.set_index("timestamp")[["temp","humidity","vibration"]].tail(300)
            chart_placeholder.line_chart(df2)
            data_table.dataframe(df.tail(10).reset_index(drop=True))
"""

with open("app.py","w") as f:
    f.write(app_code)

# Run Streamlit app via ngrok
from pyngrok import ngrok
import os

port = 8501
public_url = ngrok.connect(port)
print(f"Streamlit public URL: {public_url}")

os.system(f"streamlit run app.py --server.port {port} --server.headless true")


ERROR:pyngrok.process.ngrok:t=2025-09-18T10:01:14+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-09-18T10:01:14+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-09-18T10:01:14+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [6]:
!streamlit run app.py & npx localtunnel --port 8501

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼your url is: https://pretty-jeans-train.loca.lt
^C


In [19]:
# app.py
"""
Digital Twin Hackathon Demo App
Features:
 - Simple login (operator / manager)
 - Live simulated sensor stream (start / stop)
 - Virtual control panel (setpoints affect simulated values)
 - Upload CSV / JSON to feed dashboard instead of simulation
 - Role-based UI: operator vs manager
 - Downloadable batch reports (manager)
"""

import streamlit as st
import pandas as pd
import numpy as np
import time
from io import StringIO

# -------------------------
# --------- CONFIG --------
# -------------------------
USERS = {
    "operator": {"password": "op123", "role": "operator"},
    "manager": {"password": "mg123", "role": "manager"}
}

SIM_BATCH_SIZE = 20  # number of records per batch (batch_id increments every SIM_BATCH_SIZE rows)

# -------------------------
# ---- Helper functions ---
# -------------------------
def authenticate(username: str, password: str):
    user = USERS.get(username)
    if user and user["password"] == password:
        return {"username": username, "role": user["role"]}
    return None

def init_session():
    if "logged_in" not in st.session_state:
        st.session_state.logged_in = False
        st.session_state.user = None
    if "data" not in st.session_state:
        st.session_state.data = pd.DataFrame(columns=["timestamp","temp","humidity","vibration","batch_id"])
    if "running" not in st.session_state:
        st.session_state.running = False
    if "sim_index" not in st.session_state:
        st.session_state.sim_index = 0
    if "setpoint_temp" not in st.session_state:
        st.session_state.setpoint_temp = 60.0
        st.session_state.setpoint_humidity = 50.0
        st.session_state.setpoint_vibration = 0.5

def generate_record(i, setpoints):
    """Simulate one sensor record influenced by setpoints."""
    base_temp = setpoints["temp"]
    base_hum = setpoints["humidity"]
    base_vib = setpoints["vibration"]

    # add small random fluctuations, and occasional drift
    temp = np.random.normal(base_temp, 2.0) + 0.5 * np.sin(i/10)
    humidity = np.random.normal(base_hum, 1.5) + 0.2 * np.cos(i/13)
    vibration = max(0.01, np.random.normal(base_vib, 0.05) + 0.02 * np.sin(i/7))

    batch_id = f"batch_{(i // SIM_BATCH_SIZE) + 1}"
    return {
        "timestamp": pd.Timestamp.now(),
        "temp": round(float(temp), 2),
        "humidity": round(float(humidity), 2),
        "vibration": round(float(vibration), 3),
        "batch_id": batch_id
    }

def append_record(rec):
    df = st.session_state.data
    df = pd.concat([df, pd.DataFrame([rec])], ignore_index=True)
    st.session_state.data = df

def load_uploaded_file(uploaded_file):
    try:
        file_ext = uploaded_file.name.split(".")[-1].lower()
        if file_ext in ("csv", "txt"):
            df = pd.read_csv(uploaded_file)
        elif file_ext in ("json",):
            df = pd.read_json(uploaded_file)
        else:
            st.error("Unsupported file type. Please upload CSV or JSON.")
            return None

        # Expect at least columns: timestamp,temp,humidity,vibration,batch_id
        # If timestamp missing, create one.
        if "timestamp" not in df.columns:
            df["timestamp"] = pd.Timestamp.now()
        required = ["timestamp","temp","humidity","vibration"]
        for c in required:
            if c not in df.columns:
                st.error(f"Uploaded file missing required column: {c}")
                return None

        # Normalize columns and types
        df = df[["timestamp","temp","humidity","vibration"]].copy()
        if "batch_id" in uploaded_file.name:  # optional way to specify batches via filename
            df["batch_id"] = uploaded_file.name.split(".")[0]
        else:
            # create batch ids if not present
            df["batch_id"] = [f"batch_{(i // SIM_BATCH_SIZE) + 1}" for i in range(len(df))]

        # coerce timestamp
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        return df
    except Exception as e:
        st.error(f"Failed to parse uploaded file: {e}")
        return None

# -------------------------
# ------- UI LAYOUT -------
# -------------------------
st.set_page_config(page_title="Digital Twin Dashboard", layout="wide")
init_session()

if not st.session_state.logged_in:
    st.title("🔐 Login")
    col1, col2 = st.columns(2)
    with col1:
        username = st.text_input("Username")
    with col2:
        password = st.text_input("Password", type="password")

    if st.button("Login"):
        auth = authenticate(username, password)
        if auth:
            st.session_state.logged_in = True
            st.session_state.user = auth
            st.experimental_rerun()
        else:
            st.error("Invalid username or password. Try 'operator/op123' or 'manager/mg123'.")

else:
    user = st.session_state.user
    st.sidebar.title(f"Welcome, {user['username']} ({user['role']})")
    if st.sidebar.button("Logout"):
        # clear session and rerun
        for k in list(st.session_state.keys()):
            del st.session_state[k]
        st.experimental_rerun()

    # Navigation
    page = st.sidebar.radio("Page", ["Dashboard", "Control Panel", "Upload Data", "Admin" if user["role"]=="manager" else "Info"])

    # Top-level summary
    st.markdown("## 🍪 Digital Twin — Food Process Demo")
    st.markdown("**Live mode:** Simulated data. You can upload real CSV/JSON to switch to offline mode.")

    # Right-side container for charts & table
    if page == "Dashboard":
        left, right = st.columns([3,1])

        # Controls for simulation
        with right:
            st.subheader("Simulation Controls")
            if not st.session_state.running:
                if st.button("Start Simulation"):
                    st.session_state.running = True
            else:
                if st.button("Stop Simulation"):
                    st.session_state.running = False

            st.markdown("---")
            st.write("Setpoints (affect simulation)")
            st.session_state.setpoint_temp = st.slider("Target Temp (°C)", 20.0, 120.0, st.session_state.setpoint_temp)
            st.session_state.setpoint_humidity = st.slider("Target Humidity (%)", 0.0, 100.0, st.session_state.setpoint_humidity)
            st.session_state.setpoint_vibration = st.slider("Target Vibration (g)", 0.0, 5.0, st.session_state.setpoint_vibration)
            st.markdown("---")
            st.write("Data source:")
            st.write("• If you've uploaded data, the app will display that. Otherwise the simulation runs.")
            if st.button("Clear Stored Data"):
                st.session_state.data = pd.DataFrame(columns=["timestamp","temp","humidity","vibration","batch_id"])
                st.success("Cleared stored data")

        # Main charts
        with left:
            st.subheader("Live Sensor Chart")
            chart_placeholder = st.empty()

            st.subheader("Latest readings")
            data_table = st.empty()

            # Insert uploaded data if present in session_state (upload page will fill it)
            df = st.session_state.data.copy()

            # If simulation running, generate a new record per rerun
            if st.session_state.running:
                # generate a single record per rerun (Streamlit reruns the script frequently)
                rec = generate_record(st.session_state.sim_index, {
                    "temp": st.session_state.setpoint_temp,
                    "humidity": st.session_state.setpoint_humidity,
                    "vibration": st.session_state.setpoint_vibration
                })
                append_record(rec)
                st.session_state.sim_index += 1
                # tiny pause so UI updates look natural (non-blocking enough)
                time.sleep(0.25)

            if df.empty:
                st.info("No data yet. Start simulation or upload a file.")
            else:
                # ensure timestamp is datetime and sorted
                df["timestamp"] = pd.to_datetime(df["timestamp"])
                df = df.sort_values("timestamp")
                df2 = df.set_index("timestamp")[["temp","humidity","vibration"]].tail(300)

                # show line chart
                chart_placeholder.line_chart(df2)

                # Latest table
                data_table.dataframe(df.tail(10).reset_index(drop=True))

            # Batch status: show last value per batch
            st.subheader("Batch Status")
            if not df.empty:
                last_per_batch = df.sort_values("timestamp").groupby("batch_id").tail(1)
                st.table(last_per_batch[["batch_id","temp","humidity","vibration"]].reset_index(drop=True))

    elif page == "Control Panel":
        st.subheader("Virtual Control Panel")
        st.write("This panel simulates remote adjustments to the physical process. Changes affect subsequent simulated data.")
        st.markdown("- Adjust setpoints and then Start Simulation to observe the effect in the Dashboard.")
        st.write("Current setpoints:")
        st.write(f"Temperature: {st.session_state.setpoint_temp} °C")
        st.write(f"Humidity: {st.session_state.setpoint_humidity} %")
        st.write(f"Vibration: {st.session_state.setpoint_vibration} g")

        if user["role"] != "operator":
            st.info("Control access is normally for operators. Managers can view and adjust for testing here.")

        # quick single-step controls (for demos)
        if st.button("Simulate Heat Spike (+10°C) for 10 records"):
            base_t = st.session_state.setpoint_temp
            for j in range(10):
                rec = generate_record(st.session_state.sim_index + j, {
                    "temp": base_t + 10,
                    "humidity": st.session_state.setpoint_humidity,
                    "vibration": st.session_state.setpoint_vibration
                })
                append_record(rec)
            st.session_state.sim_index += 10
            st.success("Injected heat spike (10 records)")

    elif page == "Upload Data":
        st.subheader("Upload CSV or JSON sensor data")
        st.markdown("Expected columns: timestamp, temp, humidity, vibration (optional: batch_id). If timestamp absent, it will be added.")
        uploaded_file = st.file_uploader("Upload file", type=["csv","json"])
        if uploaded_file is not None:
            df_uploaded = load_uploaded_file(uploaded_file)
            if df_uploaded is not None:
                st.session_state.data = pd.concat([st.session_state.data, df_uploaded], ignore_index=True)
                st.success(f"Loaded {len(df_uploaded)} rows from upload into session storage.")
                st.experimental_rerun()

    elif page == "Admin":
        st.subheader("Manager Analytics & Export")
        df = st.session_state.data.copy()
        if df.empty:
            st.info("No data to analyze. Start simulation or upload a file.")
        else:
            df["timestamp"] = pd.to_datetime(df["timestamp"])
            st.markdown("**Summary statistics (latest data)**")
            st.write(df[["temp","humidity","vibration"]].describe())

            # simple anomaly rule: temp > setpoint + 10 or vibration > 1.5
            df["anomaly_rule"] = ((df["temp"] > (st.session_state.setpoint_temp + 10)) | (df["vibration"] > 1.5)).astype(int)
            st.markdown("Anomaly counts by batch (rule-based)")
            anom_counts = df.groupby("batch_id")["anomaly_rule"].sum().reset_index().rename(columns={"anomaly_rule":"anomaly_count"})
            st.table(anom_counts)

            st.markdown("Download combined data (CSV)")
            csv = df.to_csv(index=False)
            st.download_button("Download CSV", data=csv, file_name="sensor_data_export.csv", mime="text/csv")

    else:
        st.subheader("Info")
        st.markdown("This demo app simulates a digital twin pipeline. Use the Upload page to test with real data. Use Control Panel to inject conditions and Dashboard to view live charts.")

    # footer: small help
    st.markdown("---")
    st.caption("Demo app: operator credentials — user: operator / pass: op123. manager — user: manager / pass: mg123")

2025-09-18 10:10:51.958 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 10:10:51.960 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 10:10:51.961 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 10:10:51.963 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 10:10:51.964 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 10:10:51.965 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 10:10:51.969 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 10:10:51.971 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [20]:
%%writefile app.py
import streamlit as st

st.title("🚀 Hackathon Demo UI")
st.sidebar.success("Navigation Panel")

role = st.sidebar.selectbox("Login as:", ["Operator", "Manager"])
st.write(f"Hello, you logged in as **{role}**")


Overwriting app.py


In [22]:
from pyngrok import ngrok
import subprocess

# Kill previous tunnels if any
ngrok.kill()

# Set your ngrok authtoken here
ngrok.set_auth_token("YOUR_NGROK_AUTH_TOKEN")

# Start Streamlit in background
process = subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501"])

# Create public URL
public_url = ngrok.connect(8501)
print("🌍 Streamlit App URL:", public_url)

ERROR:pyngrok.process.ngrok:t=2025-09-18T10:17:52+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: YOUR_NGROK_AUTH_TOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"
ERROR:pyngrok.process.ngrok:t=2025-09-18T10:17:52+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: YOUR_NGROK_AUTH_TOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"
ERROR:pyngrok.process.ngrok:t=2025-09-18T10:17:52+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: The authtoken you specified does not look like a pr

PyngrokNgrokError: The ngrok process errored on start: authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: YOUR_NGROK_AUTH_TOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n.

In [2]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.4 MB/s eta 0:00:00
